In [ ]:
!pip install langdetect Sastrawi tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 9.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=e6068a144873271184a2c61fa7585eb832538f0ce0b7283bf1b80144be9e625b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


# Library

In [ ]:
import pandas as pd
import re
import math

from tqdm import tqdm
from langdetect import detect

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# EDA

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/jhnwlkn/STKI/main/Scraping%20by%20Subjects/merged_data.csv')
df

,division,title,abstract,subjects,publication,publisher,date,keywords
0,FK1,Identifikasi Kontaminasi Bakteri \nColiform\nP...,Abstrak\nColiform\nmerupakan bakteri yang m...,AC,Jurnal Kedokteran,NaN,2013-07,NaN
1,FMIPA4,CHEMICAL STABILITY AND ADSORPTION SELECTIVITY ...,"Chemical stability, reusability, and adsorptio...",AC,Indo. J. Chem,NaN,2012,ionic imprinted; Nannochloropsis sp; adsorptio...
2,NaN,AKTIVITAS IN VITRO DAN STUDI PERBANDINGAN BEBE...,ABSTRAK\nTelah dibuat senyawa turunan dibutil-...,"AC, AI",J. Sains MIPA,NaN,2013,NaN
3,FMIPA2,Analisis Kualitas Limbah Cair Pada Kolam Anaer...,"Among the industries in Lampung, PT. Perkebuna...",AC,BIOGENESIS,NaN,2013,"anaerobic pool, liquid waste, PTPN VII"
4,FT8,EA-MDA MODEL TO RESOLVE IS\nCHARACTERISTIC PRO...,alignment between business strategy and existi...,AC,International Journal of Software Engineering ...,NaN,2013-05,NaN
...,...,...,...,...,...,...,...,...
29156,FK1,Situational Judgement Test (SJT): Alternatif M...,"Saat ini, metode seleksi masuk Fakultas Kedokt...",RZ,JURNAL KEDOKTERAN UNIVERSITAS LAMPUNG,FK Unila,2017,NaN
29157,FK1,Hubungan antara Note Taking dengan Hasil Kuis ...,Note taking adalah suatu aktivitas yang membut...,RZ,Majority,Juke Unila,2019-12,NaN
29158,FK1,Undergraduate Medical Students’Perceptions on ...,Background: The concept of feedback-seeking be...,RZ,Malaysian Journal of Medical Sciences,Universiti Sains Malaysia,2018-02-28,NaN
29159,FK1,Perbedaan Tingkat Stres pada Mahasiswa Tahap P...,Stres merupakan interaksi individu dengan ling...,RZ,JK Unila,Fakultas Kedokteran universitas Lampung,2019-03,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29161 entries, 0 to 29160
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   division     25218 non-null  object
 1   title        29161 non-null  object
 2   abstract     22098 non-null  object
 3   subjects     29066 non-null  object
 4   publication  15062 non-null  object
 5   publisher    23432 non-null  object
 6   date         22800 non-null  object
 7   keywords     2031 non-null   object
dtypes: object(8)
memory usage: 1.8+ MB


In [ ]:
df.isna().sum()

division        3943
title              0
abstract        7063
subjects          95
publication    14099
publisher       5729
date            6361
keywords       27130
dtype: int64

In [ ]:
ambiguous_data = df.duplicated().sum()
print(ambiguous_data)

5190


In [ ]:
df['abstract'].fillna('', inplace=True)
df.isna().sum()

division        3943
title              0
abstract           0
subjects          95
publication    14099
publisher       5729
date            6361
keywords       27130
dtype: int64

# Language Detection in Title and Abstract
if there is data that is full of foreign languages then separate the data because the data that will be processed next is only Indonesian.

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

df['title_language'] = df['title'].apply(detect_language)
df['abstract_language'] = df['abstract'].apply(detect_language)

In [ ]:
df_indonesian = df[(df['abstract_language'] == 'id') & (df['title_language'] == 'id')]
df_other_languages = df[(df['abstract_language'] != 'id') | (df['title_language'] != 'id')]

In [ ]:
df_indonesian.isna().sum()

division              519
title                   0
abstract                0
subjects               37
publication          2333
publisher            1107
date                  873
keywords             5410
title_language          0
abstract_language       0
dtype: int64

In [ ]:
df_indonesian['abstract']

0        Abstrak\nColiform\nmerupakan  bakteri  yang  m...
12       Merupakan surat keputusan akreditas jurnal yan...
57       Penelitian .ini .bertujuan .untuk .(1) .mengan...
81       Hasil Penilaian Sejawat Sebidang atau Peer Rev...
82       Remaja transmigran Jawa menggunakan tiga bahas...
                               ...                        
29155    Tidur yang kurang baik dalam aspek kuantitatif...
29156    Saat ini, metode seleksi masuk Fakultas Kedokt...
29157    Note taking adalah suatu aktivitas yang membut...
29159    Stres merupakan interaksi individu dengan ling...
29160    Metode pembelajaran di fakultas kedokteran ter...
Name: abstract, Length: 5478, dtype: object

# Some Abstracts contain keywords
separate the keywords and move them to the keywords column.

In [ ]:
def get_keyword(text):
  # Cari Pola Regex Keyword pada abstract
  result = re.search(r'Keyword[s]?:( \w+[,]*)+', text)

  if result:
    # Hilangkan kata-kata "Keyword" dari hasil pencarian
    result = re.sub(r'Keyword[s]?: ', '', result.group(0))
    # Pisahkan keyword berdasarkan tanda koma, dan hapus spasi berlebih
    return [x.strip().lower() for x in result.split(',')]
  else:
    return ['']

In [ ]:
def delete_keyword(text):
  result = re.sub(r'Keyword[s]?:( \w+[,]*)+', '', text)
  return result

In [ ]:
df['keywords'] = df['abstract'].apply(lambda x: get_keyword(x))
df['abstract'] = df['abstract'].apply(lambda x: delete_keyword(x))

In [ ]:
df_keywords = df
df.to_csv('/content/drive/MyDrive/STKI/Preprocessing/1. Keywords/df_with_keyword.csv')

# Raw Fulltext

In [ ]:
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29161 entries, 0 to 29160
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   division           25218 non-null  object
 1   title              29161 non-null  object
 2   abstract           29161 non-null  object
 3   subjects           29066 non-null  object
 4   publication        15062 non-null  object
 5   publisher          23432 non-null  object
 6   date               22800 non-null  object
 7   keywords           29161 non-null  object
 8   title_language     29161 non-null  object
 9   abstract_language  29161 non-null  object
dtypes: object(10)
memory usage: 2.2+ MB


In [ ]:
def createRawFulltext(data):
  return data['title'] + ' ' + data['abstract'] + ' ' + ' '.join(data['keywords'])

In [ ]:
df_keywords['fulltext_raw'] = df_keywords.apply(lambda x: createRawFulltext(x), axis=1)

In [ ]:
df_keywords.iloc[0]['fulltext_raw']

'Identifikasi Kontaminasi Bakteri \nColiform\nPada Daging Sapi Segar Yang Dijual \nDi Pasar Sekitar Kota Bandar Lampung Abstrak\nColiform\nmerupakan  bakteri  yang  memiki  habitat  normal  di  usus  manusia  dan  juga  hewan.  Oleh \nkarena  itu  bakteri \nColiform\n,  terutama \nEscherichia  coli,\nmenjadi  indikasi  d\nari  kontaminasi  fekal \npada air minum maupun makanan. Penyebaran \nColiform\ndari manusia ke manusia yang lain dapat \nterjadi melalui jalur fekal oral yaitu dengan cara  manusia memakan makanan atau minuman yang \ntelah  terkontaminasi  feses  manusia.  Infeksi \nColifo\nrm\npada  manusia  seringkali  disebabkan  oleh \nkonsumsi  makanan  produk  hewan  yang  tercemar,  misalnya  daging  dan  susu.  Penelitian  ini \ndilakukan  untuk  untuk  mengetahui  kontaminasi \nColiform\npada  daging  sapi  segar  yang  dijual  di \npasar  di  sekitar  kota  Bandar  Lampun\ng.\nPenelitian  deteksi  bakteri \nColiform\ndilakukan  melalui \npenanaman pada  media \nMacConkey\n(

In [ ]:
df_raw_full = df_keywords
df_raw_full.to_csv('/content/drive/MyDrive/STKI/Preprocessing/2. Fulltext Raw/raw_data.csv')

 # Detect Language

In [ ]:
def detect_language(text):
  try:
    src_lang = detect(text)
  except:
    src_lang = '?'
  return src_lang

In [ ]:
df_raw_full['lang'] = df_raw_full['fulltext_raw'].apply(lambda x : detect_language(x))

In [ ]:
df_lang = df_raw_full
df_lang.to_csv('/content/drive/MyDrive/STKI/Preprocessing/3. Detect Language/df_language_detected.csv')

In [ ]:
df_id = df_lang[df['lang'] == 'id']
df_other = df_lang[df['lang'] != 'id']

In [ ]:
df_id.to_csv('/content/drive/MyDrive/STKI/Preprocessing/3. Detect Language/idn_language.csv')
df_other.to_csv('/content/drive/MyDrive/STKI/Preprocessing/3. Detect Language/other_language.csv')

# Cleaning
Remove whitespace but keep numbers with whitespace such as 0.0 0.0, -0, - 0, 0%

In [ ]:
def clean_text(text):
  result = text.lower()
  result = re.sub(r'[^\w\s\.,%\-]+', ' ', result) # Remove Symbol
  result = re.sub(r'[_]+', ' ', result) # Remove Underscore
  result = re.sub(r'([\d]+)[\s]*(%)|([\d]+)[\s]*(-)[\s]*([\d]+)|(-)[\s]*([\d]+)', r'\1\2\3\4\5\6\7', result) # Remove space between digit and symbol (1 - 2, 1 %)
  result = re.sub(r'([\D])[\.,]+', r'\1 ', result) # Remove dot and comma (exceeding)
  result = re.sub(r'[\.,]+([\D])', r'\1 ', result) # Remove dot and comma (preceeding)
  result = re.sub(r'([\D])[%]+', r'\1 ', result) # Remove Percent
  result = re.sub(r'([\D])-+([\D])|-+([\D])', r'\1\2\3 ', result) # Remove Strip
  result = re.sub(r'[\s]+', ' ', result) # Remove Multiple Whitespace
  result = result.strip()
  return result

In [ ]:
df_id['fulltext'] = df_id['fulltext_raw'].apply(lambda x: clean_text(x))

<ipython-input-27-2ef6ea8b500f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id['fulltext'] = df_id['fulltext_raw'].apply(lambda x: clean_text(x))


In [ ]:
df_cleaned = df_id

In [ ]:
df_cleaned.to_csv('/content/drive/MyDrive/STKI/Preprocessing/4. Data Cleaned/data_cleaned.csv')

# Stopword
Do stopword for fulltext, create a new column with the name fulltext_clean

In [ ]:
def remove_stopword(text):
  factory = StopWordRemoverFactory()
  custom = ['mutlak',
            'era',
            'cenderung',
            'bantu',
            'paham',
            'dimana',
            'harap',
            'bahas',
            'definisi',
            'dimana',
            'dasar',
            'duga',
            'pakai',
            'pilih',
            'sesuai',
            'meliputi',
            'dituliskan',
            'dinyatakan',
            'keberlanjutan',
            'memperoleh',
            'diperoleh',
            'diharapkan',
            'mengharapkan',
            'berharap',
            'membahas',
            'dibahas',
            'didefinisikan',
            'mendefinisikan',
            'didalam',
            'kedalam',
            'memiliki',
            'dimiliki',
            'berdasarkan',
            'didasari',
            'berlandaskan',
            'diduga',
            'menduga',
            'menggambarkan',
            'digambarkan',
            'memakai',
            'dipakai',
            'memilih',
            'dipilih',
            'disesuaikan',
            'menyesuaikan',
            'kesesuaian',
            'diliputi',
            'menjabarkan',
            'dijabarkan',
            'penjabaran']
  stopwords = factory.get_stop_words()
  stopwords.extend(custom)

  result = []
  for x in  text.split():
    if x not in stopwords:
      result.append(x)

  return " ".join(result)

In [ ]:
df_cleaned['fulltext_clean'] = df_cleaned['fulltext'].apply(lambda x: remove_stopword(x))

<ipython-input-31-4bdb99f823fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['fulltext_clean'] = df_cleaned['fulltext'].apply(lambda x: remove_stopword(x))


In [ ]:
df_stopword = df_cleaned
df_stopword.to_csv('/content/drive/MyDrive/STKI/Preprocessing/5. Stopword/df_fulltext_cleaned.csv')

# Stemming
Create a new column with the name _fulltext_clean_stemming

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/STKI/Preprocessing/5. Stopword/df_fulltext_cleaned.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df

,division,title,abstract,subjects,publication,publisher,date,keywords,title_language,abstract_language,fulltext_raw,lang,fulltext,fulltext_clean
0,FK1,Identifikasi Kontaminasi Bakteri \nColiform\nP...,Abstrak\nColiform\nmerupakan bakteri yang m...,AC,Jurnal Kedokteran,NaN,2013-07,[''],id,id,Identifikasi Kontaminasi Bakteri \nColiform\nP...,id,identifikasi kontaminasi bakteri coliform pada...,identifikasi kontaminasi bakteri coliform dagi...
1,NaN,AKTIVITAS IN VITRO DAN STUDI PERBANDINGAN BEBE...,ABSTRAK\nTelah dibuat senyawa turunan dibutil-...,"AC, AI",J. Sains MIPA,NaN,2013,[''],de,id,AKTIVITAS IN VITRO DAN STUDI PERBANDINGAN BEBE...,id,aktivitas in vitro dan studi perbandingan bebe...,aktivitas in vitro studi perbandingan beberapa...
2,FP3,KAJIAN TINGKAT KECERNAAN PAKAN BUATAN YANG BER...,ABSTRAK\nPenelitian\ndilakukan untuk mengeta...,AC,Jurnal Rekayasa dan Teknologi Budidaya Perairan,NaN,2013-02,[''],en,id,KAJIAN TINGKAT KECERNAAN PAKAN BUATAN YANG BER...,id,kajian tingkat kecernaan pakan buatan yang ber...,kajian tingkat kecernaan pakan buatan berbasis...
3,FP3,PENGGUNAAN \nTEPUNG DAGING DAN TULANG\nSEBAGAI...,ABSTRAK\n‡\nP\nenelitian\ndilakukan untuk meng...,AC,Jurnal Rekayasa Teknologi dan Budidaya Perairan,NaN,2013-10,[''],en,id,PENGGUNAAN \nTEPUNG DAGING DAN TULANG\nSEBAGAI...,id,penggunaan tepung daging dan tulang sebagai al...,penggunaan tepung daging tulang alternatif sum...
4,FP1,Akreditas Jurnal Ilmu Ilmu Agribisnis 2022,Merupakan surat keputusan akreditas jurnal yan...,AC,NaN,"Direktorat Jenderal Perguruan Tinggi, Kementer...",2022-11,[''],id,id,Akreditas Jurnal Ilmu Ilmu Agribisnis 2022 Mer...,id,akreditas jurnal ilmu ilmu agribisnis 2022 mer...,akreditas jurnal ilmu ilmu agribisnis 2022 mer...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,FK1,Hubungan Pola Tidur Terhadap Hasil Ujian Akhir...,Tidur yang kurang baik dalam aspek kuantitatif...,RZ,Majority,Juke Unila,2019-12,[''],id,id,Hubungan Pola Tidur Terhadap Hasil Ujian Akhir...,id,hubungan pola tidur terhadap hasil ujian akhir...,hubungan pola tidur hasil ujian akhir blok tro...
12218,FK1,Situational Judgement Test (SJT): Alternatif M...,"Saat ini, metode seleksi masuk Fakultas Kedokt...",RZ,JURNAL KEDOKTERAN UNIVERSITAS LAMPUNG,FK Unila,2017,[''],id,id,Situational Judgement Test (SJT): Alternatif M...,id,situational judgement test sjt alternatif meto...,situational judgement test sjt alternatif meto...
12219,FK1,Hubungan antara Note Taking dengan Hasil Kuis ...,Note taking adalah suatu aktivitas yang membut...,RZ,Majority,Juke Unila,2019-12,[''],id,id,Hubungan antara Note Taking dengan Hasil Kuis ...,id,hubungan antara note taking dengan hasil kuis ...,hubungan note taking hasil kuis perkuliahan bl...
12220,FK1,Perbedaan Tingkat Stres pada Mahasiswa Tahap P...,Stres merupakan interaksi individu dengan ling...,RZ,JK Unila,Fakultas Kedokteran universitas Lampung,2019-03,[''],id,id,Perbedaan Tingkat Stres pada Mahasiswa Tahap P...,id,perbedaan tingkat stres pada mahasiswa tahap p...,perbedaan tingkat stres mahasiswa tahap profes...


In [ ]:
def stem(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  return stemmer.stem(text)

In [ ]:
for i in tqdm(range(9, 10)):
  df_final_other = df.iloc[i*500:(i+1)*500 if (i+1)*500 < df.shape[0] else df.shape[0]]
  df_final_other['text_stemmed'] = df_final_other['fulltext_clean'].apply(lambda x : stem(x))
  df_final_other.to_csv(f'/content/drive/MyDrive/STKI/Preprocessing/6. Stemming/File Partition/ke-{i+1}.csv')

  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-36-6be9169e05c0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_other['text_stemmed'] = df_final_other['fulltext_clean'].apply(lambda x : stem(x))
100%|██████████| 1/1 [14:29<00:00, 869.61s/it]


# Post-Tag Detection

In [ ]:
!pip install spacy

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/STKI/Preprocessing/6. Stemming/File Partition/ke-10.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

,division,title,abstract,subjects,publication,publisher,date,keywords,title_language,abstract_language,fulltext_raw,lang,fulltext,fulltext_clean,text_stemmed
0,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,Pandemi Corona Virus Disease 2019 (Covid-19) d...,K1,NaN,UNIVERSITAS LAMPUNG,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...
1,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,Perlindungan paten obat khususnya obat dan ata...,K1,NaN,http://lppm.unila.ac.id/,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...
2,FK1,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,Seorang perempuan datang ke rumah sakit umum d...,K1,Jurnal Medula,NaN,2016-12,[''],id,id,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,id,kekerasan dalam rumah tangga pada kasus pernik...,kekerasan rumah tangga kasus pernikahan dini s...,keras rumah tangga kasus nikah dini orang pere...
3,FH3,Angin Baru Penegakan Hukum di Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Angin Baru Penegakan Hukum di Lampung,id,angin baru penegakan hukum di lampung,angin baru penegakan hukum lampung,angin baru tega hukum lampung
4,FH3,Bumi Menangis di Bandar Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Bumi Menangis di Bandar Lampung,id,bumi menangis di bandar lampung,bumi menangis bandar lampung,bumi menang bandar lampung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,FH2,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,Secara empirik peraturan perundang-undangan (U...,K1,NaN,DPD-RI,2018-02-22,[''],de,id,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,id,penelitian empirik dalam rangka penyusunan ran...,penelitian empirik rangka penyusunan rancangan...,teliti empirik rangka susun rancang undangu nd...
496,FH2,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,Berdasarkan dua ketentuan Pasal 53 dan 54A Per...,K1,NaN,---,2019-08-16,[''],sw,id,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,id,pengadaan tanah skala kecil untuk kepentingan ...,pengadaan tanah skala kecil kepentingan umum k...,ada tanah skala kecil penting umum tentu pasal...
497,FH2,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,Pengembalian hak atas tanah milik Kongregasi S...,K1,NaN,Universitas lampung,2018-05-07,[''],de,id,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,id,pengembalian hak atas tanah masyarakat yang di...,pengembalian hak atas tanah masyarakat dipergu...,kembali hak atas tanah masyarakat guna pemda k...
498,FH3,PERJANJIAN PERKAWINAN:PERSPEKTIF KEPEMILIKAN H...,"Sampai dengan akhir tahun 2016, seorang Warga ...",K1,NaN,NaN,2017-10-17,[''],tl,id,PERJANJIAN PERKAWINAN:PERSPEKTIF KEPEMILIKAN H...,id,perjanjian perkawinan perspektif kepemilikan h...,perjanjian perkawinan perspektif kepemilikan h...,janji kawin perspektif milik hak milik atas ta...


In [ ]:
pos_tags_list = []

for text in df['fulltext_clean']:
    doc = nlp(text)

    pos_tags = [(token.text, token.pos_) for token in doc]
    pos_tags_list.append(pos_tags)

In [ ]:
df['fulltext_clean_postag'] = pos_tags_list

df.to_csv('/content/drive/MyDrive/STKI/Preprocessing/7. Post-tag Detection/df_with_pos-tag.csv', index=False)

# Synonim Data Augmentation

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00


In [ ]:
import nltk
import json
import random

import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/STKI/Preprocessing/7. Post-tag Detection/df_with_pos-tag.csv')
df

,division,title,abstract,subjects,publication,publisher,date,keywords,title_language,abstract_language,fulltext_raw,lang,fulltext,fulltext_clean,text_stemmed,fulltext_clean_postag
0,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,Pandemi Corona Virus Disease 2019 (Covid-19) d...,K1,NaN,UNIVERSITAS LAMPUNG,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...,"[('kepentingan', 'PROPN'), ('umum', 'PROPN'), ..."
1,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,Perlindungan paten obat khususnya obat dan ata...,K1,NaN,http://lppm.unila.ac.id/,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...,"[('kepentingan', 'PROPN'), ('umum', 'PROPN'), ..."
2,FK1,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,Seorang perempuan datang ke rumah sakit umum d...,K1,Jurnal Medula,NaN,2016-12,[''],id,id,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,id,kekerasan dalam rumah tangga pada kasus pernik...,kekerasan rumah tangga kasus pernikahan dini s...,keras rumah tangga kasus nikah dini orang pere...,"[('kekerasan', 'PROPN'), ('rumah', 'PROPN'), (..."
3,FH3,Angin Baru Penegakan Hukum di Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Angin Baru Penegakan Hukum di Lampung,id,angin baru penegakan hukum di lampung,angin baru penegakan hukum lampung,angin baru tega hukum lampung,"[('angin', 'VERB'), ('baru', 'NOUN'), ('penega..."
4,FH3,Bumi Menangis di Bandar Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Bumi Menangis di Bandar Lampung,id,bumi menangis di bandar lampung,bumi menangis bandar lampung,bumi menang bandar lampung,"[('bumi', 'PROPN'), ('menangis', 'PROPN'), ('b..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,FH2,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,Secara empirik peraturan perundang-undangan (U...,K1,NaN,DPD-RI,2018-02-22,[''],de,id,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,id,penelitian empirik dalam rangka penyusunan ran...,penelitian empirik rangka penyusunan rancangan...,teliti empirik rangka susun rancang undangu nd...,"[('penelitian', 'PROPN'), ('empirik', 'PROPN')..."
496,FH2,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,Berdasarkan dua ketentuan Pasal 53 dan 54A Per...,K1,NaN,---,2019-08-16,[''],sw,id,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,id,pengadaan tanah skala kecil untuk kepentingan ...,pengadaan tanah skala kecil kepentingan umum k...,ada tanah skala kecil penting umum tentu pasal...,"[('pengadaan', 'PROPN'), ('tanah', 'PROPN'), (..."
497,FH2,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,Pengembalian hak atas tanah milik Kongregasi S...,K1,NaN,Universitas lampung,2018-05-07,[''],de,id,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,id,pengembalian hak atas tanah masyarakat yang di...,pengembalian hak atas tanah masyarakat dipergu...,kembali hak atas tanah masyarakat guna pemda k...,"[('pengembalian', 'ADJ'), ('hak', 'PROPN'), ('..."
498,FH3,PERJANJIAN PERKAWINAN:PERSPEKTIF KEPEMILIKAN H...,"Sampai dengan akhir tahun 2016, seorang Warga ...",K1,NaN,NaN,2017-10-17,[''],tl,id,PERJANJIAN PERKAWINAN:PERSPEKTIF KEPEMILIKAN H...,id,perjanjian perkawinan perspektif kepemilikan h...,perjanjian perkawinan perspektif kepemilikan h...,janji kawin perspektif milik hak milik atas ta...,"[('perjanjian', 'PROPN'), ('perkawinan', 'PROP..."


In [ ]:
def replace_text(text, unmasker, count):
  input_text = text

  orig_text_list = input_text.split()

  for i in range(10):
    len_input = len(orig_text_list)
    new_text_list = orig_text_list.copy()
    if len_input>2:
      rand_idx = random.randint(1,len_input-1)

      orig_word = orig_text_list[rand_idx]

      new_text_list[rand_idx] = '[MASK]'

      new_mask_sent = ' '.join(new_text_list)

      augmented_text_list = unmasker(new_mask_sent)

      for res in augmented_text_list:
        if res['token_str'] != orig_word:
          augmented_text = res['sequence']
          orig_text_list = augmented_text.split()
          break
    else:
      augmented_text = ' '.join(orig_text_list)

  return augmented_text

In [ ]:
from transformers import pipeline

unmasker = pipeline('fill-mask', model='cahya/distilbert-base-indonesian')
print(df['fulltext_clean'].iloc[0])
print(replace_text(df['fulltext_clean'].iloc[0], unmasker, 10))

kepentingan umum pemberian paten obatdi indonesia menyoal pemberian paten vaksin obat covid-19 pandemi corona virus disease 2019 covid-19 kebutuhan global mendesak segera mendapatkan obat vaksin covid-19 efektif berdampak perlombaan penemuanp enemuan farmasi pendaftaran paten obat vaksin covid 19 fenomena menuai pro kontra timbul kekhawatiran benturan kepentingan ekonomi pemegang paten kepentingan umum masyarakat luas monopoli paten penemu perusahaan farmasi ditakutkan membuat negara kesulitan menyediakan obat vaksin masyarakat produksi peredaran obat tersebut berada sepenuhnya penguasaan pemegang paten diskusi akademik intens dilakukan menentukan landasan normative menyeimbangan kepentingan berbeda kepentingan ekonomi pemegang kepentingan umum masyarakat melalui penyediaan keterjangkauan obat covid-19
kepentingan umum pemberian paten obatdi indonesia menyoal pemberian paten vaksin obat covid - 19 pandemi corona virus disease pandemi covid - 19 kebutuhan global mendesak segera mendapat

In [ ]:
df_augment = df.copy()

def apply_replace_text(x):
    return replace_text(x, unmasker, 10)

df_augment['fulltext_clean_synonim'] = df_augment['fulltext_clean'].apply(apply_replace_text)

In [ ]:
df_augment

,division,title,abstract,subjects,publication,publisher,date,keywords,title_language,abstract_language,fulltext_raw,lang,fulltext,fulltext_clean,text_stemmed,fulltext_clean_postag,fulltext_clean_synonim
0,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,Pandemi Corona Virus Disease 2019 (Covid-19) d...,K1,NaN,UNIVERSITAS LAMPUNG,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...,"[('kepentingan', 'PROPN'), ('umum', 'PROPN'), ...",kepentingan mendesak pemberian paten obatdi in...
1,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,Perlindungan paten obat khususnya obat dan ata...,K1,NaN,http://lppm.unila.ac.id/,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...,"[('kepentingan', 'PROPN'), ('umum', 'PROPN'), ...",kepentingan umum pemberian paten obatdi indone...
2,FK1,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,Seorang perempuan datang ke rumah sakit umum d...,K1,Jurnal Medula,NaN,2016-12,[''],id,id,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,id,kekerasan dalam rumah tangga pada kasus pernik...,kekerasan rumah tangga kasus pernikahan dini s...,keras rumah tangga kasus nikah dini orang pere...,"[('kekerasan', 'PROPN'), ('rumah', 'PROPN'), (...",kekerasan rumah tangga kasus pernikahan dini s...
3,FH3,Angin Baru Penegakan Hukum di Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Angin Baru Penegakan Hukum di Lampung,id,angin baru penegakan hukum di lampung,angin baru penegakan hukum lampung,angin baru tega hukum lampung,"[('angin', 'VERB'), ('baru', 'NOUN'), ('penega...",angin kencang dapat terjadi
4,FH3,Bumi Menangis di Bandar Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Bumi Menangis di Bandar Lampung,id,bumi menangis di bandar lampung,bumi menangis bandar lampung,bumi menang bandar lampung,"[('bumi', 'PROPN'), ('menangis', 'PROPN'), ('b...",bumi bandang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,FH2,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,Secara empirik peraturan perundang-undangan (U...,K1,NaN,DPD-RI,2018-02-22,[''],de,id,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,id,penelitian empirik dalam rangka penyusunan ran...,penelitian empirik rangka penyusunan rancangan...,teliti empirik rangka susun rancang undangu nd...,"[('penelitian', 'PROPN'), ('empirik', 'PROPN')...",penelitian empirik rangka menyusun rancangan u...
496,FH2,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,Berdasarkan dua ketentuan Pasal 53 dan 54A Per...,K1,NaN,---,2019-08-16,[''],sw,id,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,id,pengadaan tanah skala kecil untuk kepentingan ...,pengadaan tanah skala kecil kepentingan umum k...,ada tanah skala kecil penting umum tentu pasal...,"[('pengadaan', 'PROPN'), ('tanah', 'PROPN'), (...",pengadaan lahan pembangunan untuk kepentingan ...
497,FH2,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,Pengembalian hak atas tanah milik Kongregasi S...,K1,NaN,Universitas lampung,2018-05-07,[''],de,id,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,id,pengembalian hak atas tanah masyarakat yang di...,pengembalian hak atas tanah masyarakat dipergu...,kembali hak atas tanah masyarakat guna pemda k...,"[('pengembalian', 'ADJ'), ('hak', 'PROPN'), ('...",pengembalian hak atas tanah masyarakat kepada ...
498,FH3,PERJANJIAN PERKAWINAN:PERSPEKTIF KEPEMILIKAN H...,"Sampai dengan akhir tahun 2016, seorang Warga ...",K1,NaN,NaN,2017-10-17,[''],tl,id,PERJANJIAN PERKAWINAN:PERSPEKTIF KEPEMILIKAN H...,id,perjanjian perkawinan perspektif kepemilikan h...,perjanjian perkawinan perspektif kepemilikan h...,janji kawin perspektif milik hak milik atas ta...,"[('perjanjian', 'PROPN'),

In [ ]:
df_augment.to_csv('/content/drive/MyDrive/STKI/Preprocessing/8. Data Augmentation/synonims.csv')

# Data Augmentation

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
def delete_random(word):
  token = word_tokenize(word)
  random_element = random.choice(token)
  token.remove(random_element)

  return ' '.join(token)

In [ ]:
df_augment_random = df_augment
df_augment_random['fulltext_clean_augment_deletion'] = df_augment_random['fulltext_clean'].apply(lambda x: delete_random(x))
df_augment_random

,division,title,abstract,subjects,publication,publisher,date,keywords,title_language,abstract_language,fulltext_raw,lang,fulltext,fulltext_clean,text_stemmed,fulltext_clean_postag,fulltext_clean_synonim,fulltext_clean_augment_deletion
0,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,Pandemi Corona Virus Disease 2019 (Covid-19) d...,K1,NaN,UNIVERSITAS LAMPUNG,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN\nOBATDI...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...,"[('kepentingan', 'PROPN'), ('umum', 'PROPN'), ...",kepentingan mendesak pemberian paten obatdi in...,kepentingan umum pemberian paten obatdi indone...
1,FH3,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,Perlindungan paten obat khususnya obat dan ata...,K1,NaN,http://lppm.unila.ac.id/,2021,[''],tl,id,KEPENTINGAN UMUM DALAM PEMBERIAN PATEN OBATDI ...,id,kepentingan umum dalam pemberian paten obatdi ...,kepentingan umum pemberian paten obatdi indone...,penting umum beri paten obatdi indonesia soal ...,"[('kepentingan', 'PROPN'), ('umum', 'PROPN'), ...",kepentingan umum pemberian paten obatdi indone...,kepentingan umum pemberian paten obatdi indone...
2,FK1,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,Seorang perempuan datang ke rumah sakit umum d...,K1,Jurnal Medula,NaN,2016-12,[''],id,id,Kekerasan dalam Rumah Tangga pada Kasus Pernik...,id,kekerasan dalam rumah tangga pada kasus pernik...,kekerasan rumah tangga kasus pernikahan dini s...,keras rumah tangga kasus nikah dini orang pere...,"[('kekerasan', 'PROPN'), ('rumah', 'PROPN'), (...",kekerasan rumah tangga kasus pernikahan dini s...,kekerasan rumah tangga kasus pernikahan dini s...
3,FH3,Angin Baru Penegakan Hukum di Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Angin Baru Penegakan Hukum di Lampung,id,angin baru penegakan hukum di lampung,angin baru penegakan hukum lampung,angin baru tega hukum lampung,"[('angin', 'VERB'), ('baru', 'NOUN'), ('penega...",angin kencang dapat terjadi,angin baru hukum lampung
4,FH3,Bumi Menangis di Bandar Lampung,NaN,"K1, KZ",Kolom Pakar harian Lampung Post,NaN,NaN,[''],id,unknown,Bumi Menangis di Bandar Lampung,id,bumi menangis di bandar lampung,bumi menangis bandar lampung,bumi menang bandar lampung,"[('bumi', 'PROPN'), ('menangis', 'PROPN'), ('b...",bumi bandang,menangis bandar lampung
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,FH2,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,Secara empirik peraturan perundang-undangan (U...,K1,NaN,DPD-RI,2018-02-22,[''],de,id,PENELITIAN EMPIRIK DALAM RANGKA PENYUSUNAN RAN...,id,penelitian empirik dalam rangka penyusunan ran...,penelitian empirik rangka penyusunan rancangan...,teliti empirik rangka susun rancang undangu nd...,"[('penelitian', 'PROPN'), ('empirik', 'PROPN')...",penelitian empirik rangka menyusun rancangan u...,penelitian empirik rangka penyusunan rancangan...
496,FH2,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,Berdasarkan dua ketentuan Pasal 53 dan 54A Per...,K1,NaN,---,2019-08-16,[''],sw,id,PENGADAAN TANAH SKALA KECIL UNTUK KEPENTINGAN ...,id,pengadaan tanah skala kecil untuk kepentingan ...,pengadaan tanah skala kecil kepentingan umum k...,ada tanah skala kecil penting umum tentu pasal...,"[('pengadaan', 'PROPN'), ('tanah', 'PROPN'), (...",pengadaan lahan pembangunan untuk kepentingan ...,pengadaan tanah skala kecil kepentingan umum k...
497,FH2,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,Pengembalian hak atas tanah milik Kongregasi S...,K1,NaN,Universitas lampung,2018-05-07,[''],de,id,PENGEMBALIAN HAK ATAS TANAH MASYARAKAT \nYANG ...,id,pengembalian hak atas tanah masyarakat yang di...,pengembalian hak atas tanah masyarakat dipergu...,kembali hak atas tanah masyarakat guna pemda k...,"[('pengembalian', 'ADJ'), ('hak', 'PROPN'), ('...",pengembalian hak atas tanah masyarakat kepada ...,pengembalian hak atas tanah masyaraka

In [ ]:
df_augment_random.to_csv('/content/drive/MyDrive/STKI/Preprocessing/8. Data Augmentation/random_deletion.csv', index=False)